# Explore AirBNB Dataset and Using DiskANN with Python

In [ ]:
%pip install -r ../app/requirements.txt

In [5]:
from openai import AzureOpenAI
import os
import pgvector
import numpy as np
import psycopg
from pgvector.psycopg import register_vector
import json


## Load necessary environment variables

In [6]:
%load_ext dotenv
%dotenv

In [7]:
CONN_STR = os.getenv("AZURE_PG_CONNECTION")
DEFAULT_SEARCH_QUERY = "homes near beach that allows pets and parking"

## Define Function for embedding queries

In [8]:
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
	)

def embedding_query(text_input):
	# Generate embedding for each text input
	response = client.embeddings.create(
		input=text_input,
		model="text-embedding-3-small"  # Use the appropriate model
	)

	# Parse the JSON string
	json_response = response.model_dump_json(indent=2)
	parsed_response = json.loads(json_response)

	# Extract the embedding key
	embedding = parsed_response['data'][0]['embedding']
	return embedding

## Simple vector search on database

In [9]:
import psycopg

# Define the SQL query
query = """
SELECT name, description 
		FROM listings_search
		ORDER BY description_vector <=> %s::vector
        LIMIT 10;
"""

# Connect to the PostgreSQL database
with psycopg.connect(CONN_STR) as conn:
    with conn.cursor() as cur:
        # Execute the query
        emb = embedding_query(DEFAULT_SEARCH_QUERY)
        
        cur.execute(query, (emb,))
        
        # Fetch the results
        results = cur.fetchall()
        
        # Print the results
        for row in results:
            print(row)

('Seattle Pet/Family Friendly Living', 'Diverse city location, conveniently 1block to bus line, nearby grocery/coffee shop. Easy access to I-5 & shopping! Furnishings throughout, detailed cleaning for your comfort.Owner on premise & available!We love letting well behaved/clean pets \\stay\\! The entire space is for your privacy, a dedicated parking spot for your vehicle. Leave your car and take the bus if you choose. * We hire professional staffing to clean the unit after each guest visits, every area and place the guest touches is sanitized and cleaned for the safety and cleanliness and peace of mind when you stay with us. All sheets and linens are laundered each and every time a guest leaves.  The Oaktree Suite is on the ground level perfect for your cute little doggie to roam outside the enclosed patio area. The kitchen has full appliances for your stay and have a Blender, Juicer, George Foreman Grill, salad spinner, toaster, microwave, coffee maker, rice cooker, lots of pots and pa

## Vector Search with filters - Using No Index

In [14]:
sql_query = """
	WITH listing_cte AS
	(
		SELECT l.listing_id, name, summary, description, description_vector
		FROM listings l 
		INNER JOIN calendar c ON l.listing_id = c.listing_id 
		WHERE c.date = '2017-01-02' AND c.available = 't' AND c.price < 30
	)
	SELECT l.listing_id, name, summary, description
	FROM listing_cte l
	ORDER BY description_vector <=> %s::vector
	LIMIT 10;
"""

# Establish a connection to the PostgreSQL database
with psycopg.connect(CONN_STR) as conn:
	with conn.cursor() as cur:
		# Execute the SQL query
		emb = embedding_query(DEFAULT_SEARCH_QUERY)
		
		print("="*10, "Explanation of the query", "="*10)
		explain = "explain analyze"
		cur.execute(explain+sql_query, (emb,))
		
		# Fetch the results
		results = cur.fetchall()
		
		# Print the results
		for row in results:
			print(row)
		print("="*10, "Rows from Database", "="*10)
		# Fetch the results
		cur.execute(sql_query, (emb,))
		results = cur.fetchall()
		
		# Print the results
		for row in results:
			print(row)
		
		retrieved_ids_no_index = [row[0] for row in results]

========== Explanation of the query ==========
('Limit  (cost=1787.20..2469.88 rows=10 width=1115) (actual time=2929.278..3114.194 rows=10 loops=1)',)
('  ->  Nested Loop  (cost=1787.20..33600.21 rows=466 width=1115) (actual time=2929.276..3114.184 rows=10 loops=1)',)
('        Join Filter: (l.listing_id = c.listing_id)',)
('        Rows Removed by Join Filter: 20818',)
('        ->  Index Scan using listings_search_cosine_diskann on listings l  (cost=787.20..2977.44 rows=2224 width=1125) (actual time=2199.861..3001.680 rows=2083 loops=1)',)
("              Order By: (description_vector <=> '[0.0068256045,0.00038570366,0.06252902,0.04077353,-0.022247748,-0.062144816,-0.016676806,0.017697345,0.020086607,0.022007622,0.022055646,-0.0027749653,-0.01312293,0.025549492,-0.04363104,-0.013759266,0.004649455,0.0038090113,0.057678457,0.007443931,0.019342214,-0.022764022,-0.04579218,-0.044399444,0.0075399815,0.00037294693,-0.053068023,-0.019330207,0.042262316,-0.0077380864,0.008158308,-0.03935678

In [15]:
set(retrieved_ids_no_index)

{2459519,
 3666970,
 3811828,
 3994601,
 3994634,
 4701205,
 4825073,
 7011773,
 7064681,
 7411863}

## Vector Search with filters - Using HNSW index

In [24]:
sql_query = """
	WITH listing_cte AS
	(
		SELECT l.listing_id, c.price, name, summary, description, description_vector
		FROM listings_hnsw l 
		INNER JOIN calendar c ON l.listing_id = c.listing_id 
		WHERE c.date = '2017-01-02' AND c.available = 't' AND c.price < 30
	)
	SELECT l.listing_id, l.price, name, summary, description
	FROM listing_cte l
	ORDER BY description_vector <=> %s::vector
	LIMIT 1000; -- Brute force at around 200
"""
# Establish a connection to the PostgreSQL database
with psycopg.connect(CONN_STR) as conn:
	with conn.cursor() as cur:
		# Execute the SQL query
		emb = embedding_query(DEFAULT_SEARCH_QUERY)

		print("="*10, "Explanation of the query", "="*10)
		
		explain = "explain analyze"
		off_scan = "SET LOCAL enable_seqscan TO OFF;"
		ef_search = "SET LOCAL hnsw.ef_search = 1000;" #Max is 1000 https://jkatz05.com/post/postgres/pgvector-overview-0.5.0/ 

		cur.execute(off_scan)
		cur.execute(ef_search)
		cur.execute(explain+sql_query, (emb,))
		
		# Fetch the results
		results = cur.fetchall()
		
		# Print the results
		for row in results:
			print(row)
		print("="*10, "Rows from Database", "="*10)


		# Fetch the non explained results
		cur.execute(sql_query, (emb,))
		results = cur.fetchall()
		
		# Print the results
		for row in results:
			print(row)
		
		retrieved_hnsw_ids = [row[0] for row in results]

========== Explanation of the query ==========
('Limit  (cost=10000000128.48..10000040247.51 rows=466 width=1120) (actual time=75.443..75.971 rows=1 loops=1)',)
('  ->  Nested Loop  (cost=10000000128.48..10000040247.51 rows=466 width=1120) (actual time=75.442..75.968 rows=1 loops=1)',)
('        Join Filter: (l.listing_id = c.listing_id)',)
('        Rows Removed by Join Filter: 10009',)
('        ->  Index Scan using listing_cosine_hnsw on listings_hnsw l  (cost=128.48..1202.72 rows=2224 width=1125) (actual time=6.520..7.140 rows=1001 loops=1)',)
("              Order By: (description_vector <=> '[0.0068256045,0.00038570366,0.06252902,0.04077353,-0.022247748,-0.062144816,-0.016676806,0.017697345,0.020086607,0.022007622,0.022055646,-0.0027749653,-0.01312293,0.025549492,-0.04363104,-0.013759266,0.004649455,0.0038090113,0.057678457,0.007443931,0.019342214,-0.022764022,-0.04579218,-0.044399444,0.0075399815,0.00037294693,-0.053068023,-0.019330207,0.042262316,-0.0077380864,0.008158308,-0.03

In [18]:
set(retrieved_hnsw_ids)

{7064681}

## Recall for HNSW vs No Index
Both for HNSW with no metadata filter and with metadata filter. Recall is the same because the HNSW index can only retrieve max 1000 entries. In the max 1000 entries, their is only 1 result that satisfies the filter. However this table has 2000+ rows, the other 4 are not top 1000 in the relevency results

In [19]:
len(set(retrieved_hnsw_ids).intersection(set(retrieved_ids_no_index))) / len(set(retrieved_ids_no_index))

0.1

## Vector Search with filters - Using DiskANN index

With iterator

In [21]:
sql_query = """
	WITH listing_cte AS
	(
		SELECT l.listing_id, name, summary, description, description_vector
		FROM listings_diskann l 
		INNER JOIN calendar c ON l.listing_id = c.listing_id 
		WHERE c.date = '2017-01-02' AND c.available = 't' AND c.price < 30
	)
	SELECT l.listing_id, name, summary, description
	FROM listing_cte l
	ORDER BY description_vector <=> %s::vector
	LIMIT 10;
"""

# Establish a connection to the PostgreSQL database
with psycopg.connect(CONN_STR) as conn:
	with conn.cursor() as cur:
		# Execute the SQL query
		emb = embedding_query(DEFAULT_SEARCH_QUERY)
		
		print("="*10, "Explanation of the query", "="*10)

		explain = "explain analyze"
		off_scan = "SET LOCAL enable_seqscan TO OFF;"

		cur.execute(off_scan)

		cur.execute(explain+sql_query, (emb,))
		
		# Fetch the results
		results = cur.fetchall()
		
		# Print the results
		for row in results:
			print(row)
		print("="*10, "Rows from Database", "="*10)

		# Fetch the results
		cur.execute(sql_query, (emb,))
		results = cur.fetchall()
		
		# Print the results
		for row in results:
			print(row)
		retrieved_ids_diskann = [row[0] for row in results]


========== Explanation of the query ==========
('Limit  (cost=10000000479.20..10000001332.44 rows=10 width=1175) (actual time=72.520..75.263 rows=10 loops=1)',)
('  ->  Nested Loop  (cost=10000000479.20..10000040240.23 rows=466 width=1175) (actual time=72.519..75.260 rows=10 loops=1)',)
('        Join Filter: (l.listing_id = c.listing_id)',)
('        Rows Removed by Join Filter: 20876',)
('        ->  Index Scan using listing_cosine_diskann on listings_diskann l  (cost=479.20..1195.44 rows=2224 width=1185) (actual time=1.389..4.485 rows=2089 loops=1)',)
("              Order By: (description_vector <=> '[0.0068256045,0.00038570366,0.06252902,0.04077353,-0.022247748,-0.062144816,-0.016676806,0.017697345,0.020086607,0.022007622,0.022055646,-0.0027749653,-0.01312293,0.025549492,-0.04363104,-0.013759266,0.004649455,0.0038090113,0.057678457,0.007443931,0.019342214,-0.022764022,-0.04579218,-0.044399444,0.0075399815,0.00037294693,-0.053068023,-0.019330207,0.042262316,-0.0077380864,0.00815830

In [22]:
set(retrieved_ids_diskann)

{2459519,
 3666970,
 3811828,
 3994601,
 3994634,
 4701205,
 4825073,
 7011773,
 7064681,
 7411863}

## Recall for DiskANN vs No Index
This works because of the iterator return. We keep increasing the candidate list until we run through the index or we reach to asked top k

In [23]:
len(set(retrieved_ids_diskann).intersection(set(retrieved_ids_no_index))) / len(set(retrieved_ids_no_index))

1.0